In [2]:
import pygame
import numpy as np

class DrawingMatrix:
    def __init__(self, grid_size=28, window_size=560):
        self.grid_size = grid_size
        self.window_size = window_size
        self.block_size = window_size // grid_size
        
        # Initialize pygame
        pygame.init()
        self.window = pygame.display.set_mode((window_size, window_size))
        pygame.display.set_caption("Draw and Extract Matrix")
        
        # Colors
        self.BG_COLOR = (255, 255, 255)  # White background
        self.DRAW_COLOR = (0, 0, 0)      # Black drawing
        self.GRID_COLOR = (200, 200, 200) # Light gray grid
        
        # Initialize grid
        self.grid_array = []
        self.setup_grid()
        
        # State
        self.running = True
        
    def setup_grid(self):
        """Initialize the grid with white rectangles"""
        self.grid_array = []
        for i in range(self.grid_size):
            for j in range(self.grid_size):
                rect = pygame.Rect(i * self.block_size, j * self.block_size, 
                                 self.block_size, self.block_size)
                color = self.BG_COLOR
                self.grid_array.append((rect, color))
    
    def get_matrix(self):
        """Convert the current drawing to a numpy matrix"""
        matrix = np.zeros((self.grid_size, self.grid_size))
        
        for row in range(self.grid_size):
            for col in range(self.grid_size):
                index = col * self.grid_size + row
                rect, color = self.grid_array[index]
                
                # If the cell is black (drawn), set value to 0.9
                if color == self.DRAW_COLOR:
                    matrix[row, col] = 0.9
        
        return matrix
    
    def clear_grid(self):
        """Clear the entire grid"""
        for i in range(len(self.grid_array)):
            rect, _ = self.grid_array[i]
            self.grid_array[i] = (rect, self.BG_COLOR)
    
    def draw_grid(self):
        """Draw the grid and cells"""
        for rect, color in self.grid_array:
            pygame.draw.rect(self.window, color, rect)
            pygame.draw.rect(self.window, self.GRID_COLOR, rect, 1)
    
    def print_matrix(self):
        """Print the current matrix to console"""
        matrix = self.get_matrix()
        print("\nCurrent Matrix:")
        print(matrix)
        print(f"Shape: {matrix.shape}")
        print(f"Non-zero elements: {np.count_nonzero(matrix)}")
    
    def run(self):
        """Main drawing loop"""
        clock = pygame.time.Clock()
        
        print("Drawing Matrix Extractor")
        print("- Click and drag to draw")
        print("- Press SPACE to clear")
        print("- Press ENTER to print matrix")
        print("- Press ESC to exit")
        
        while self.running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.running = False
                    
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        self.running = False
                    elif event.key == pygame.K_SPACE:
                        self.clear_grid()
                        print("Grid cleared")
                    elif event.key == pygame.K_RETURN:
                        self.print_matrix()
            
            # Handle drawing
            if pygame.mouse.get_pressed()[0]:  # Left mouse button
                mouse_pos = pygame.mouse.get_pos()
                for index, (rect, color) in enumerate(self.grid_array):
                    if rect.collidepoint(mouse_pos):
                        self.grid_array[index] = (rect, self.DRAW_COLOR)
            
            # Draw everything
            self.window.fill(self.BG_COLOR)
            self.draw_grid()
            pygame.display.flip()
            clock.tick(60)
        
        # Print final matrix when exiting
        print("\nFinal matrix:")
        final_matrix = self.get_matrix()
        print(final_matrix)
        
        pygame.quit()
        return final_matrix


# Example usage
if __name__ == "__main__":
    drawer = DrawingMatrix()
    matrix = drawer.run()

Drawing Matrix Extractor
- Click and drag to draw
- Press SPACE to clear
- Press ENTER to print matrix
- Press ESC to exit

Final matrix:
[[0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.9 0.9 0.9 0.9 0.9 0.9 0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.9 0.  0.  0.  0.  0.  0.9 0.9 0.9
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.9 0.  0.  0.  0.  0.  0.  0.  0.  0.9
  0.9 0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.9 0.  0.  0.

In [3]:
import pygame
import numpy as np
from scipy import ndimage

class DrawingMatrixPreprocessed:
    def __init__(self, grid_size=28, window_size=560):
        self.grid_size = grid_size
        self.window_size = window_size
        self.block_size = window_size // grid_size
        
        # Initialize pygame
        pygame.init()
        self.window = pygame.display.set_mode((window_size, window_size))
        pygame.display.set_caption("Draw and Extract Matrix (with Preprocessing)")
        
        # Colors
        self.BG_COLOR = (255, 255, 255)  # White background
        self.DRAW_COLOR = (0, 0, 0)      # Black drawing
        self.GRID_COLOR = (200, 200, 200) # Light gray grid
        
        # Initialize grid
        self.grid_array = []
        self.setup_grid()
        
        # State
        self.running = True
        
    def setup_grid(self):
        """Initialize the grid with white rectangles"""
        self.grid_array = []
        for i in range(self.grid_size):
            for j in range(self.grid_size):
                rect = pygame.Rect(i * self.block_size, j * self.block_size, 
                                 self.block_size, self.block_size)
                color = self.BG_COLOR
                self.grid_array.append((rect, color))
    
    def get_raw_matrix(self):
        """Convert the current drawing to a raw numpy matrix"""
        matrix = np.zeros((self.grid_size, self.grid_size))
        
        for row in range(self.grid_size):
            for col in range(self.grid_size):
                index = col * self.grid_size + row
                rect, color = self.grid_array[index]
                
                # If the cell is black (drawn), set value to 0.9
                if color == self.DRAW_COLOR:
                    matrix[row, col] = 0.9
        
        return matrix
    
    def center_and_normalize_digit(self, digit_array, target_size=20):
        """Center and normalize digit to fit within target size"""
        rows, cols = np.where(digit_array > 0)
        
        if len(rows) == 0:
            return digit_array
        
        min_row, max_row = rows.min(), rows.max()
        min_col, max_col = cols.min(), cols.max()
        
        digit_region = digit_array[min_row:max_row+1, min_col:max_col+1]
        
        height, width = digit_region.shape
        scale = min(target_size / height, target_size / width)
        
        if scale < 1.0:
            new_height = int(height * scale)
            new_width = int(width * scale)
            digit_region = ndimage.zoom(digit_region, (new_height/height, new_width/width))
        
        centered_digit = np.zeros((28, 28))
        
        region_height, region_width = digit_region.shape
        start_row = (28 - region_height) // 2
        start_col = (28 - region_width) // 2
        
        centered_digit[start_row:start_row+region_height, start_col:start_col+region_width] = digit_region
        
        return centered_digit

    def calculate_center_of_mass(self, digit_array):
        """Calculate center of mass of the digit"""
        rows, cols = np.where(digit_array > 0)
        if len(rows) == 0:
            return 14, 14
        
        weights = digit_array[rows, cols]
        center_row = np.average(rows, weights=weights)
        center_col = np.average(cols, weights=weights)
        
        return center_row, center_col

    def fine_tune_centering(self, digit_array):
        """Fine-tune centering using center of mass"""
        center_row, center_col = self.calculate_center_of_mass(digit_array)
        
        target_center = 13.5
        shift_row = int(round(target_center - center_row))
        shift_col = int(round(target_center - center_col))
        
        shift_row = max(-10, min(10, shift_row))
        shift_col = max(-10, min(10, shift_col))
        
        if shift_row != 0 or shift_col != 0:
            digit_array = ndimage.shift(digit_array, (shift_row, shift_col), cval=0.0)
        
        return digit_array

    def preprocess_drawing(self, digit_array):
        """Apply full preprocessing pipeline"""
        # Step 1: Center and normalize
        centered = self.center_and_normalize_digit(digit_array)
        
        # Step 2: Fine-tune centering
        final = self.fine_tune_centering(centered)
        
        # Step 3: Normalize for neural network input
        normalized = (final - 0.5) / 0.5
        
        return final, normalized
    
    def clear_grid(self):
        """Clear the entire grid"""
        for i in range(len(self.grid_array)):
            rect, _ = self.grid_array[i]
            self.grid_array[i] = (rect, self.BG_COLOR)
    
    def draw_grid(self):
        """Draw the grid and cells"""
        for rect, color in self.grid_array:
            pygame.draw.rect(self.window, color, rect)
            pygame.draw.rect(self.window, self.GRID_COLOR, rect, 1)
    
    def print_matrices(self):
        """Print both raw and preprocessed matrices"""
        raw_matrix = self.get_raw_matrix()
        
        if np.sum(raw_matrix) == 0:
            print("\nNo drawing detected!")
            return
        
        preprocessed, normalized = self.preprocess_drawing(raw_matrix)
        
        print("\n" + "="*50)
        print("RAW MATRIX:")
        print("="*50)
        print(raw_matrix)
        print(f"Shape: {raw_matrix.shape}")
        print(f"Non-zero elements: {np.count_nonzero(raw_matrix)}")
        
        print("\n" + "="*50)
        print("PREPROCESSED MATRIX (after centering):")
        print("="*50)
        print(preprocessed)
        print(f"Shape: {preprocessed.shape}")
        print(f"Non-zero elements: {np.count_nonzero(preprocessed)}")
        
        print("\n" + "="*50)
        print("NORMALIZED MATRIX (neural network input):")
        print("="*50)
        print(normalized)
        print(f"Shape: {normalized.shape}")
        print(f"Range: [{normalized.min():.3f}, {normalized.max():.3f}]")
    
    def run(self):
        """Main drawing loop"""
        clock = pygame.time.Clock()
        
        print("Drawing Matrix Extractor with Preprocessing")
        print("- Click and drag to draw")
        print("- Press SPACE to clear")
        print("- Press ENTER to print matrices (raw + preprocessed)")
        print("- Press ESC to exit")
        
        while self.running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.running = False
                    
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        self.running = False
                    elif event.key == pygame.K_SPACE:
                        self.clear_grid()
                        print("Grid cleared")
                    elif event.key == pygame.K_RETURN:
                        self.print_matrices()
            
            # Handle drawing
            if pygame.mouse.get_pressed()[0]:  # Left mouse button
                mouse_pos = pygame.mouse.get_pos()
                for index, (rect, color) in enumerate(self.grid_array):
                    if rect.collidepoint(mouse_pos):
                        self.grid_array[index] = (rect, self.DRAW_COLOR)
            
            # Draw everything
            self.window.fill(self.BG_COLOR)
            self.draw_grid()
            pygame.display.flip()
            clock.tick(60)
        
        # Print final matrices when exiting
        print("\nFinal matrices:")
        self.print_matrices()
        
        pygame.quit()


# Example usage
if __name__ == "__main__":
    drawer = DrawingMatrixPreprocessed()
    drawer.run()

Drawing Matrix Extractor with Preprocessing
- Click and drag to draw
- Press SPACE to clear
- Press ENTER to print matrices (raw + preprocessed)
- Press ESC to exit

RAW MATRIX:
[[0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.9 0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.9 0.9 0.9 0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.9 0.9 0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.9 0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  